In [1]:
# Download dataset

import requests
import os

if not 'shakespeare.txt' in os.listdir('./'):
    r = requests.get('https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
    open('shakespeare.txt', 'wb').write(r.content)

In [2]:
import numpy as np

# For every character create distinct ID and encode the text
n_chars = 0

with open('shakespeare.txt', 'r') as f:
    char2ids = {}
    idx2char = {}
    encoded = []
    for l in f.readlines():
        for char in l:
            n_chars += 1
            if char not in char2ids.keys():
                char2ids[char] = len(char2ids)
                idx2char[len(char2ids)-1] = char
            encoded.append(char2ids[char])    
    encoded = np.array(encoded, dtype=np.int8)
    
print(char2ids)
print(idx2char)

{'F': 0, 'i': 1, 'r': 2, 's': 3, 't': 4, ' ': 5, 'C': 6, 'z': 7, 'e': 8, 'n': 9, ':': 10, '\n': 11, 'B': 12, 'f': 13, 'o': 14, 'w': 15, 'p': 16, 'c': 17, 'd': 18, 'a': 19, 'y': 20, 'u': 21, 'h': 22, ',': 23, 'm': 24, 'k': 25, '.': 26, 'A': 27, 'l': 28, 'S': 29, 'Y': 30, 'v': 31, '?': 32, 'R': 33, 'M': 34, 'W': 35, "'": 36, 'L': 37, 'I': 38, 'N': 39, 'g': 40, ';': 41, 'b': 42, '!': 43, 'O': 44, 'j': 45, 'V': 46, '-': 47, 'T': 48, 'H': 49, 'E': 50, 'U': 51, 'D': 52, 'P': 53, 'q': 54, 'x': 55, 'J': 56, 'G': 57, 'K': 58, 'Q': 59, '&': 60, 'Z': 61, 'X': 62, '3': 63, '$': 64}
{0: 'F', 1: 'i', 2: 'r', 3: 's', 4: 't', 5: ' ', 6: 'C', 7: 'z', 8: 'e', 9: 'n', 10: ':', 11: '\n', 12: 'B', 13: 'f', 14: 'o', 15: 'w', 16: 'p', 17: 'c', 18: 'd', 19: 'a', 20: 'y', 21: 'u', 22: 'h', 23: ',', 24: 'm', 25: 'k', 26: '.', 27: 'A', 28: 'l', 29: 'S', 30: 'Y', 31: 'v', 32: '?', 33: 'R', 34: 'M', 35: 'W', 36: "'", 37: 'L', 38: 'I', 39: 'N', 40: 'g', 41: ';', 42: 'b', 43: '!', 44: 'O', 45: 'j', 46: 'V', 47: '-',

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class ShakespeareDataset(Dataset):
    def __init__(self, encoded, n_unique_chars, window_size=101):
        self.n_unique_chars = n_unique_chars
        self.encoded = torch.tensor(encoded, dtype=torch.float)
        self.window_size = window_size
        
    def __len__(self):
        return len(self.encoded) - self.window_size - 1
    
    def __getitem__(self, idx):
        window = self.encoded[idx:idx+self.window_size]
        return window[:-1].long(), window[1:].long()

In [4]:
train_size = int(n_chars * 0.9) # Take 90% of text as training data
print(train_size)
train_dataset = ShakespeareDataset(encoded[:train_size], len(char2ids))
train_dataset[1][0].shape

1003854


torch.Size([100])

In [5]:
batch_size = 1
loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
device = torch.device('cuda' if not torch.cuda.is_available() else 'cpu')

In [8]:
# Model

class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embed = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, X, hidden):
        X = self.embed(X)
        X, hidden = self.gru(X.unsqueeze(1), hidden)
        X = self.out(X.reshape(X.shape[0], -1))
        return X, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, X, hidden, cell):
        print(X.shape)
        out = self.embed(X)
        print(out.shape)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        print(out.shape)
        out = self.fc(out.reshape(out.shape[0], -1))
        print(out.shape)
        input()
        return out, (hidden, cell)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell

In [9]:
model = CharRNN(len(char2ids), 256, 2, len(char2ids)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
check_every = 100

def generate(model, init_str="Ty", prediction_len=200, temperature=0.85):
    hidden = model.init_hidden(batch_size)
    init_inp = torch.tensor([char2ids[char] for char in init_str]).long()
    pred = init_str

    for p in range(len(init_str) - 1):
        _, hidden = model(init_inp[p].view(1).to(device), hidden)

    last_char = init_inp[-1]

    for p in range(prediction_len):
        output, hidden = model(last_char.view(1).to(device), hidden)
        output_distance = output.data.view(-1).div(temperature).exp()
        top_char = torch.multinomial(output_distance, 1)[0]
        predicted_char = idx2char[int(top_char)]
        pred += predicted_char
        last_char = torch.tensor(char2ids[predicted_char])

    return pred

for epoch in range(10):
    for j, (window, target) in enumerate(loader):
        model.zero_grad()
        hidden = model.init_hidden(batch_size)
        optimizer.zero_grad()
        window = window.to(device)
        target = target.to(device)
        loss = 0
        
        for i in range(window.shape[1]): 
            out, hidden = model(window[:,i], hidden)  
            loss += criterion(out, target[:,i])

        if j%check_every == 0:
            print('+=+++++++++')
            print(generate(model))
            print("+++++++++++++++++")
            
        loss.backward()
        optimizer.step()
        print(f'\r {loss.item() / window.shape[1]}', end='')

+=+++++++++
Tyjoy$pXx'D$myCgja'Rq?UMPLV
&;li,L!fjWJw3cJVM,V
iI!pxr
cnqx?BLwD;bDq3qxlMjT uOKCNKBFa;?Ls,AEIGx
!zuT&xYxlJiGixE'TIVJNwkj:Gqcaofk'COBl:vJg;WYl&Mr3W?BU:cUfbFv'qqkpveLE-qGBRKcYiE3begVUgPw!sOKzulUSZF'dn
ELe
+++++++++++++++++
 2.603546447753906+=+++++++++
Ty beare wie;
I INdd me your weard veou ave theat the inr am?

Tuste the I Burish.

UU:
I dan; ounc ararl the od mind doughu
Wiml ord woh a dout af, are be;
R Ey oour shae qwavot go hit los mllorey mang
+++++++++++++++++
 2.1876789855957033+=+++++++++
Ty leke ir to her wey.f
Yithe,, conte.

OMISCE:
Cull were butre with spepenteresllou whe kny thur with nout prits wor or, mrang oraveed
The for sleePosd wuor! my;
Thaver and her, Sralk; moun a apcher'.

+++++++++++++++++
 1.9981024169921875+=+++++++++
Ty firse though in paeld
Shas me as bow man hin the on sners, shis the and knop, shave not, gordsanse.

VINALA:
I degtiren
Pos in and me this this soume ad nese bing as sand the shid an thour you dich'd
+++++++++++++++++
 2.087711486

KeyboardInterrupt: 